### Decorator Application: Decorating Classes

До сих пор мы работали с декорированием функций. Это значит, что мы можем декорировать функции, определенные с помощью оператора `def` (мы можем использовать синтаксис `@` или длинную форму). Поскольку методы класса являются функциями, их тоже можно декорировать. Лямбда-выражения также можно декорировать (используя длинную форму).

Но если задуматься о том, как работают наши декораторы, то они принимают один параметр, функцию, и возвращают какую-то другую функцию — обычно замыкание, которое использует исходную функцию, переданную в качестве аргумента.

Мы могли бы использовать ту же концепцию, чтобы принять не функцию, а класс. Мы могли бы ссылаться на этот класс внутри нашего декоратора, изменять его, а затем возвращать этот измененный класс.

Сначала мы рассмотрим то, что называется **monkey patching**. Это сводится к изменению или расширению нашего кода во время выполнения**.

Например, мы можем изменять или добавлять атрибуты к классам во время выполнения. Модули тоже.

В Python многие из используемых нами классов могут быть изменены во время выполнения
(встроенные классы, такие как строки, списки и т. д., не могут).

Но классы, написанные на Python, такие как те, которые мы пишем, и даже библиотечные классы, если они написаны на Python, а не на C, могут. Например, `Fraction` в модуле `fractions` может быть подвергнут monkey patching.

Однако то, что мы можем что-то сделать, не означает, что мы должны! Monkey patching может быть чрезвычайно полезным, но не делайте этого только потому, что можете — как всегда, должна быть реальная причина для этого, как мы увидим немного позже.

Кроме того, в целом плохая идея - патчить специальные методы `__???__` (например, `__len__`), так как это часто не работает из-за того, как эти методы ищутся в Python.

In [1]:
from fractions import Fraction

In [2]:
Fraction.speak = lambda self: 'This is a late parrot.'

In [3]:
f = Fraction(2, 3)

In [4]:
f

Fraction(2, 3)

In [5]:
f.speak()

'This is a late parrot.'

Да, это, очевидно, чепуха, но вы понимаете, что можете добавлять атрибуты к классам, даже если у вас нет прямого контроля над классом или после того, как ваш класс был определен.

Если вам нужен более полезный метод, как насчет того, который сообщает нам, является ли дробь целым числом? (т. е. знаменатель равен `1`)

In [6]:
Fraction.is_integral = lambda self: self.denominator == 1

In [7]:
f1 = Fraction(1, 2)
f2 = Fraction(10, 5)

In [8]:
f1.is_integral()

False

In [9]:
f2.is_integral()

True

Теперь мы можем внести это изменение в класс, вызвав функцию, которая сделает это:

In [10]:
def dec_speak(cls):
    cls.speak = lambda self: 'This is a very late parrot.'
    return cls

In [11]:
Fraction = dec_speak(Fraction)

_(Надеюсь, приведенный выше код напомнит вам декораторы.)_

In [12]:
f = Fraction(10, 2)

In [13]:
f.speak()

'This is a very late parrot.'

Мы также можем использовать эту функцию для декорирования наших пользовательских классов, используя короткий синтаксис **@**.

In [14]:
@dec_speak
class Parrot:
    def __init__(self):
        self.state = 'late'

In [15]:
polly = Parrot()

In [16]:
polly.speak()

'This is a very late parrot.'

Используя эту технику, мы могли бы, например, добавить полезный атрибут *reciprocal* к классу Fraction, но, конечно, поскольку это, вероятно, будет одноразовым (сколько классов Fraction, в которые вы захотите добавить обратный элемент, в конце концов), нет необходимости в декораторах. Декораторы полезны, когда их можно повторно использовать более общими способами.

In [17]:
Fraction.recip = lambda self: Fraction(self.denominator, self.numerator)

In [18]:
f = Fraction(2,3)

In [19]:
f

Fraction(2, 3)

In [20]:
f.recip()

Fraction(3, 2)

Эти примеры довольно тривиальны и не очень полезны.

Так зачем же поднимать эту тему?

Потому что эту же технику можно использовать для более интересных вещей.

В качестве первого примера предположим, что вы обычно хотите проверить различные свойства объекта в целях отладки, например, адрес памяти, его текущее состояние (значения свойств) и время, когда была сгенерирована отладочная информация.

In [21]:
from datetime import datetime, timezone

In [22]:
def debug_info(cls):
    def info(self):
        results = []
        results.append('time: {0}'.format(datetime.now(timezone.utc)))
        results.append('class: {0}'.format(self.__class__.__name__))
        results.append('id: {0}'.format(hex(id(self))))

        if vars(self):
            for k, v in vars(self).items():
                results.append('{0}: {1}'.format(k, v))

        # we have not covered lists, the extend method and generators,
        # but note that a more Pythonic way to do this would be:
        #if vars(self):
        #    results.extend('{0}: {1}'.format(k, v)
        #                   for k, v in vars(self).items())

        return results

    cls.debug = info

    return cls

In [23]:
@debug_info
class Person:
    def __init__(self, name, birth_year):
        self.name = name
        self.birth_year = birth_year

    def say_hi():
        return 'Hello there!'

In [24]:
p1 = Person('John', 1939)

In [25]:
p1.debug()

['time: 2018-02-09 04:44:02.893951+00:00',
 'class: Person',
 'id: 0x2dfe29a4630',
 'name: John',
 'birth_year: 1939']

И, конечно, мы можем декорировать таким же образом и другие классы, а не только один класс:

In [26]:
@debug_info
class Automobile:
    def __init__(self, make, model, year, top_speed_mph):
        self.make = make
        self.model = model
        self.year = year
        self.top_speed_mph = top_speed_mph
        self.current_speed = 0

    @property
    def speed(self):
        return self.current_speed

    @speed.setter
    def speed(self, new_speed):
        self.current_speed = new_speed

In [27]:
s = Automobile('Ford', 'Model T', 1908, 45)

In [28]:
s.debug()

['time: 2018-02-09 04:44:03.562898+00:00',
 'class: Automobile',
 'id: 0x2dfe29b3a58',
 'make: Ford',
 'model: Model T',
 'year: 1908',
 'top_speed_mph: 45',
 'current_speed: 0']

In [29]:
s.speed = 20

In [30]:
s.debug()

['time: 2018-02-09 04:44:03.898085+00:00',
 'class: Automobile',
 'id: 0x2dfe29b3a58',
 'make: Ford',
 'model: Model T',
 'year: 1908',
 'top_speed_mph: 45',
 'current_speed: 20']

In [31]:
from math import sqrt

In [32]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __abs__(self):
        return sqrt(self.x**2 + self.y**2)

    def __repr__(self):
        return 'Point({0},{1})'.format(self.x, self.y)

In [33]:
p1, p2, p3 = Point(2, 3), Point(2, 3), Point(0,0)

In [34]:
abs(p1)

3.605551275463989

In [35]:
p1, p2

(Point(2,3), Point(2,3))

In [36]:
p1 == p2

False

Хм, мы, вероятно, ожидали, что `p1` будет равен `p2`, поскольку у него те же координаты. Но по умолчанию Python будет сравнивать адреса памяти, поскольку наш класс не реализует метод `__eq__`, используемый для сравнений `==`.

In [37]:
p2, p3

(Point(2,3), Point(0,0))

In [38]:
p2 > p3

TypeError: '>' not supported between instances of 'Point' and 'Point'

Таким образом, этот класс не поддерживает операторы сравнения, такие как `<`, `<=` и т. д.

Даже `==` не работает так, как ожидается — он будет использовать адрес памяти вместо сравнения координат `x` и `y`, как мы могли бы ожидать.

Для оператора `<` нам нужен наш класс для реализации метода `__lt__`, а для `==` нам нужен метод `__eq__`.

Другие операторы сравнения поддерживаются путем реализации различных функций, таких как `__le__` (`<=`), `__gt__` (`>`), `__ge__` (`>=`).

Мы собираемся добавить методы `__lt__` и `__eq__` в наш класс Point.

Мы будем считать объект Point меньшим другого, если он находится ближе к началу координат (т. е. имеет меньшую величину).

In [39]:
del Point

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __abs__(self):
        return sqrt(self.x**2 + self.y**2)

    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return NotImplemented

    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented

    def __repr__(self):
        return '{0}({1},{2})'.format(self.__class__.__name__, self.x, self.y)

In [40]:
p1, p2, p3 = Point(2, 3), Point(2, 3), Point(0,0)

In [41]:
p1, p2, p1==p2

(Point(2,3), Point(2,3), True)

In [42]:
p2, p3, p2==p3

(Point(2,3), Point(0,0), False)

As we can see, `==` now works as expected

In [43]:
p4 = Point(1, 2)

In [44]:
abs(p1), abs(p4), p1 < p4

(3.605551275463989, 2.23606797749979, False)

Отлично, теперь у нас реализованы `<` и `==`. А как насчет остальных операторов: `<=`, `>`, `>=`?

In [45]:
p1 > p4

True

О, поскольку мы реализовали `<` и `==`, означает ли это, что Python волшебным образом реализовал оператор `>` (т. е. не < и не ==)?

Не совсем так! Произошло следующее: поскольку `p1` и `p4` являются точками, выполнение сравнения `p1 > p4` на самом деле то же самое, что и оценка `p4 < p1` - и Python сделал это автоматически для нас.

Но он не реализовал ни одно из других, например `>=` и `<=`:

In [46]:
p1 <= p4

TypeError: '<=' not supported between instances of 'Point' and 'Point'

Теперь, хотя мы могли бы действовать аналогичным образом и определить `>=`, `<=` и `>`, используя ту же технику, обратите внимание, что если определены `<` и `==`, то:

* `a <= b` тогда и только тогда, когда `a < b или a == b`
* `a > b` тогда и только тогда, когда `not(a<b) and a != b`
* `a >= b` тогда и только тогда, когда `not(a<b)`

Итак, чтобы быть достаточно общими, мы могли бы создать декоратор, который будет реализовывать эти последние три оператора, пока определены `==` и `<`. Затем мы могли бы декорировать **любой** класс, реализующий только эти два оператора.

In [47]:
def complete_ordering(cls):
    if '__eq__' in dir(cls) and '__lt__' in dir(cls):
        cls.__le__ = lambda self, other: self < other or self == other
        cls.__gt__ = lambda self, other: not(self < other) and not (self == other)
        cls.__ge__ = lambda self, other: not (self < other)
    return cls

На самом деле, код выше **НЕ** является хорошей реализацией вообще. Мы не проверяем совместимость типов и не возвращаем результат `NotImplemented`, если это уместно. Я также использую встроенные операторы (`<` и `==`) вместо функций dunder (`__lt__` и `__eq__`). Я просто упростил его, потому что чуть позже мы воспользуемся лучшей альтернативой.

Например, лучший способ реализовать `__ge__` будет следующим:

In [48]:
def ge_from_lt(self, other):
    # self >= other iff not(other < self)
    result = self.__lt__(other)
    if result is NotImplemented:
        return NotImplemented
    else:
        return not result

Вы можете задаться вопросом, почему я использовал `__lt__` вместо того, чтобы просто использовать оператор `<`. Это потому, что я хочу фактически посмотреть на результат операции, не вызывая исключения, если операция не реализована. Способ, которым я реализовал декоратор общего порядка, может привести к бесконечному циклу, потому что когда я оцениваю `self < other`, если возникает исключение, Python отразит оценку в `other > self`, и если это также вызовет ошибку, Python попытается отразить и эту операцию, и мы попадаем в бесконечный цикл (который в конечном итоге завершается переполнением стека). На самом деле это была ошибка в стандартной библиотечной реализации Python декоратора `complete_ordering` (называемого `total_ordering`), которая была устранена в версии 3.4.

In [49]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __abs__(self):
        return sqrt(self.x**2 + self.y**2)

    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return NotImplemented

    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented

    def __repr__(self):
        return '{0}({1},{2})'.format(self.__class__, self.x, self.y)

In [50]:
Point = complete_ordering(Point)

In [51]:
p1, p2, p3 = Point(1, 1), Point(3, 4), Point(3, 4)

In [52]:
abs(p1), abs(p2), abs(p3)

(1.4142135623730951, 5.0, 5.0)

In [53]:
p1 < p2, p1 <= p2, p1 > p2, p1 >= p2, p2 > p2, p2 >= p3

(True, True, False, False, False, True)

Теперь декоратор `complete_ordering` можно также напрямую применить к любому классу, который определяет `__eq__` и `__lt__`.

In [54]:
@complete_ordering
class Grade:
    def __init__(self, score, max_score):
        self.score = score
        self.max_score = max_score
        self.score_percent = round(score / max_score * 100)

    def __repr__(self):
        return 'Grade({0}, {1})'.format(self.score, self.max_score)

    def __eq__(self, other):
        if isinstance(other, Grade):
            return self.score_percent == other.score_percent
        else:
            return NotImplemented

    def __lt__(self, other):
        if isinstance(other, Grade):
            return self.score_percent < other.score_percent
        else:
            return NotImplemented


In [55]:
g1 = Grade(10, 100)
g2 = Grade(20, 30)
g3 = Grade(5, 50)

In [56]:
g1 <= g2, g1 == g3, g2 > g3

(True, True, True)

Часто, если задан оператор `==` и только **один** из других операторов сравнения (`<`, `<=`, `>`, `>=`), то все остальное может быть выведено.

Наш декоратор настаивал на `==` и `<`. но мы могли бы сделать его лучше, настаивая на `==` и любом другом операторе. Это, конечно, усложнит наш декоратор, и на самом деле, в Python есть эта точная функциональность, встроенная в, как вы уже догадались, модуль `functools`!

Это декоратор, который называется `total_ordering`.

Давайте посмотрим на него в действии:

In [57]:
from functools import total_ordering

In [58]:
@total_ordering
class Grade:
    def __init__(self, score, max_score):
        self.score = score
        self.max_score = max_score
        self.score_percent = round(score / max_score * 100)

    def __repr__(self):
        return 'Grade({0}, {1})'.format(self.score, self.max_score)

    def __eq__(self, other):
        if isinstance(other, Grade):
            return self.score_percent == other.score_percent
        else:
            return NotImplemented

    def __lt__(self, other):
        if isinstance(other, Grade):
            return self.score_percent < other.score_percent
        else:
            return NotImplemented

In [59]:
g1, g2 = Grade(80, 100), Grade(60, 100)

In [60]:
g1 >= g2, g1 > g2

(True, True)

Или мы могли бы сделать это следующим образом:

In [61]:
@total_ordering
class Grade:
    def __init__(self, score, max_score):
        self.score = score
        self.max_score = max_score
        self.score_percent = round(score / max_score * 100)

    def __repr__(self):
        return 'Grade({0}, {1})'.format(self.score, self.max_score)

    def __eq__(self, other):
        if isinstance(other, Grade):
            return self.score_percent == other.score_percent
        else:
            return NotImplemented

    def __gt__(self, other):
        if isinstance(other, Grade):
            return self.score_percent > other.score_percent
        else:
            return NotImplemented

In [62]:
g1, g2 = Grade(80, 100), Grade(60, 100)

In [63]:
g1 >= g2, g1 > g2, g1 <= g2, g1 < g2

(True, True, False, False)

---